In [9]:
import keras
from keras.layers import Input, UpSampling2D, Flatten, BatchNormalization, Dense, Dropout, GlobalAveragePooling2D
from keras import optimizers
from keras.datasets import cifar100
from keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.models import Sequential
from skimage.transform import resize
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet50 import ResNet50, preprocess_input

In [4]:
num_classes = 100
nb_epochs = 10

(x_train, y_train), (x_test, y_test) = cifar100.load_data()

#Pre-process the data
x_train = preprocess_input(x_train)
x_test = preprocess_input(x_test)
#Keras ImageDataGenerator provides a quick and easy way to augment your images. like standardization, rotation, shifts, flips, brightness change, and many more
datagen = ImageDataGenerator()
datagen.fit(x_train)

169001437/169001437 [==============================] - 2s 0us/step


In [5]:
#y_train initially: [0, 1] After to_categorical with num_classes=3: y_train becomes: [[1, 0, 0], [0, 1, 0]]
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

In [14]:
# weights are taken based on the imagenet ( a pretrained model) we can take weights = None if want to train from stractch
# include_top= False which makes us design our dense newtork
resnet_model = ResNet50(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

for layer in resnet_model.layers:
    if isinstance(layer, BatchNormalization):
        layer.trainable = True
    else:
        layer.trainable = False

model = Sequential()
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(UpSampling2D())
model.add(resnet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(256, activation='relu'))
model.add(Dropout(.25))
model.add(BatchNormalization())
model.add(Dense(num_classes, activation='softmax'))

In [15]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
import time
t=time.time()
model.fit(x=datagen.flow(x_train, y_train, batch_size=1),
          steps_per_epoch=len(x_train),
          epochs=5,
          validation_data=(x_test, y_test))

print('Training time: %s' % (t - time.time()))

Epoch 1/5
10480/50000 [=====>........................] - ETA: 7:16:17 - loss: 4.6160 - accuracy: 0.0118

In [ ]:
X = preprocess_input(x_test)
preds = model.predict(X)
print(preds)
